The below code will read raw data and chunk ready for loading into vector store and knowledge graph.

In [52]:
file = "data/raw/pdf/Geocoding via LINZ Address Matching.pdf"

In [71]:
import os

# Get the file name without extension and replace the extension with "txt"
txt_file = os.path.join("data/processed", os.path.splitext(os.path.basename(file))[0] + ".txt")
print(txt_file)

data/processed/Geocoding via LINZ Address Matching.txt


# llmsherpa

In [53]:
# from llmsherpa.readers import LayoutPDFReader

# # from llama_index.readers.schema.base import Document
# from llama_index.core.schema import Document
# from llama_index.core import VectorStoreIndex

In [54]:
# llmsherpa_api_url = "http://localhost:5010/api/parseDocument?renderFormat=all&applyOcr=yes"
# pdf_reader = LayoutPDFReader(llmsherpa_api_url)
# doc = pdf_reader.read_pdf(file)

In [55]:
# for chunk in doc.sections():
#     print(chunk.to_text(include_children=True, recurse=True))

# pymupdf4llm

In [69]:
import pymupdf4llm
from common import create_folder, clear_folder

img_dir = "data/processed/pymupdf4llm/images"
create_folder(img_dir)
clear_folder(img_dir)

data = pymupdf4llm.to_markdown(file, write_images=True, page_chunks=True, image_path=img_dir)

Folder 'data/processed/pymupdf4llm/images' has been created (or already exists).
Contents of the folder 'data/processed/pymupdf4llm/images' have been deleted.
Processing data/raw/pdf/Geocoding via LINZ Address Matching.pdf...
[                                        ] (0/17[                                        ] (  1/17[                                        ] (  2/17[                                        ] (  3/17[                                        ] (  4/176[=                                       ] (  5/176[=                                       ] (  6/176[=                                       ] (  7/176[=                                       ] (  8/17[==                                      ] (  9/17[==                                      ] ( 10/17[==                                      ] ( 11/17[==                                      ] ( 12/17[==                                      ] ( 13/176[===                                     ] ( 14/176[===                

In [73]:

path = "data/processed/pymupdf4llm/txt"
create_folder(path)
clear_folder(path)
output_file = os.path.join(path, os.path.splitext(os.path.basename(file))[0] + ".txt")
print(txt_file)

with open(output_file, "w") as f:
    for i, row in enumerate(data):
        f.write(f"Page: {row['metadata']['page']}\n")
        f.write(f"{row['text']}\n")
        f.write("\n")  # Add a new line after each entry

Folder 'data/processed/pymupdf4llm/txt' has been created (or already exists).
Contents of the folder 'data/processed/pymupdf4llm/txt' have been deleted.
data/processed/Geocoding via LINZ Address Matching.txt


# PyMuPDF

In [66]:
import pymupdf

doc = pymupdf.open(file)

In [ ]:
path = "data/processed/pymupdf/txt"
create_folder(path)
clear_folder(path)
output_file = os.path.join(path, os.path.splitext(os.path.basename(file))[0] + ".txt")
print(txt_file)

with open(output_file, "w") as f:
    for i, row in enumerate(doc):
        f.write(row)
        f.write(row.get_text().encode("utf8"))
        f.write("\n")  # Add a new line after each entry

Folder 'data/processed/pymupdf/txt' has been created (or already exists).
Contents of the folder 'data/processed/pymupdf/txt' have been deleted.
data/processed/Geocoding via LINZ Address Matching.txt


TypeError: write() argument must be str, not Page